In [1]:
import pandas as pd
import json
import numpy as np
from sklearn.model_selection import train_test_split
from simplet5 import *
import os
from utils import *

ddir = '../../data/'
dname = 'complete'
nbname = 't5-m4'
outputdir="/scratch/userid/" + nbname + '/'
if not os.path.exists(outputdir):
    os.makedirs(outputdir)
resultsfile = 'results/' + nbname + '.csv'


Global seed set to 42


In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=3


env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=3


In [3]:
df_train = pd.read_csv(ddir + dname + '_train.csv')
df_test = pd.read_csv(ddir + dname + '_test.csv')
df_val = pd.read_csv(ddir + dname + '_val.csv')

In [4]:
def preprocess_df_m4(df):
    X = []
    y = []
    ids = []
    for index, row in df.iterrows():
        try:
            tmp = ""
            tmp = "<claim> " + row['claim'] + "\n" if isinstance(row['claim'], str) else "<relation> "
            tmp += "<context> " + row['context'] + "\n" if isinstance(row['context'], str) else "<context> "
            tmp2 = "<mismatch> " + row['Mismatch']
            X.append(tmp)
            y.append(tmp2)
            ids.append(row['id'])
        except:
            continue
    return X,y,ids

In [5]:
X_train, y_train, id_train = preprocess_df_m4(df_train)
X_val, y_val, id_val = preprocess_df_m4(df_val)
X_test, y_test, id_test = preprocess_df_m4(df_test)

In [6]:
len(X_train), len(X_val), len(X_test)

(6443, 806, 806)

In [7]:
train_df = pd.DataFrame({'source_text': X_train, 'target_text': y_train, 'id': id_train})
val_df = pd.DataFrame({'source_text': X_val, 'target_text': y_val, 'id': id_val})
test_df = pd.DataFrame({'source_text': X_test, 'target_text': y_test, 'id': id_test})

In [8]:
train_df

source_text  \
0     <claim> Ederson Moraes plays only as a forward...   
1     <claim> A Song of Ice and Fire is by someone w...   
2     <claim> Beautiful couldn't chart on the Billbo...   
3     <claim> Jacqueline Kennedy Onassis is forgotte...   
4     <claim> Nikola Tesla didn't receive an educati...   
...                                                 ...   
6438  <claim> Inhumans is a movie.\n<context> Marvel...   
6439  <claim> Indian was solely produced by Mel Gibs...   
6440  <claim> Elle Fanning is not an actress.\n<cont...   
6441  <claim> The New Adventures of Old Christine is...   
6442  <claim> Chokher Bali was nominated for Best Ac...   

                                            target_text    id  
0                            <mismatch> as a goalkeeper  2145  
1     <mismatch> by the American novelist and screen...  1719  
2                       <mismatch> peaked at number two  1079  
3                              <mismatch> is remembered  5824  
4                                   <mismatch> received   712  
...                                                 ...   ...  
6438  <mismatch> an upcoming American television series  4719  
6439                        <mismatch> by A. M. Rathnam  3022  
6440                                      <mismatch> is  5872  
6441           <mismatch> an American television sitcom  7052  
6442  <mismatch> the Golden Leopard -LRB- Best Film ...  5926  

[6443 rows x 3 columns]

In [9]:
# load model
model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-small")

/home/userid/anaconda/envs/p3/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [10]:
# train model
delete_whole_folder(outputdir)
model.train(train_df=train_df,
            eval_df=val_df, 
            source_max_token_len=256, 
            target_max_token_len=64, 
            batch_size=16, 
            max_epochs=4, 
            outputdir = outputdir,
            )
            


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [3]
Set SLURM handle signals.

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 60.5 M
-----------------------------------------------------
60.5 M    Trainable params
0         Non-trainable params
60.5 M    Total params
242.026   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/home/userid/anaconda/envs/p3/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 40 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
Global seed set to 42
/home/userid/anaconda/envs/p3/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 40 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [11]:
import os, fnmatch
def find(pattern, path):
    result = []
    for root, dirs, files in os.walk(path):
        for name in dirs:
            if fnmatch.fnmatch(name, pattern):
                result.append(os.path.join(root, name))
    return result
find('simplet5-epoch-3*', outputdir)[0]

'/scratch/userid/t5-m4/simplet5-epoch-3-train-loss-0.1232-val-loss-0.1164'

In [12]:
from tqdm import tqdm
# load the best model

last_epoch_model = outputdir + find('simplet5-epoch-3*', outputdir)[0] # put the name here
model.load_model("t5-small", last_epoch_model, use_gpu=True)
# model.eval()
# for each test data perform prediction
predictions = []
for index, row in tqdm(test_df.iterrows()):
    prediction = model.predict(row['source_text'])[0]
    predictions.append(prediction)


# computer performance
test_df['predicted_label'] = predictions
test_df['original_label'] = test_df['target_text']

806it [01:30,  8.92it/s]


In [13]:
test_df

source_text  \
0    <claim> John Stewart is a comic book villain.\...   
1    <claim> Jimmy Carter served as President of Co...   
2    <claim> Christopher Lee portrayed Hillary Clin...   
3    <claim> Richard III of England died in the Bat...   
4    <claim> George H. W. Bush served as the 39th p...   
..                                                 ...   
801  <claim> Denial (2016 film) was released in the...   
802  <claim> Star Wars: The Force Awakens received ...   
803  <claim> Muhammad was raised under the care of ...   
804  <claim> Haim Saban is exclusively an Italian-A...   
805  <claim> Nikola Tesla refused to be a physicist...   

                                           target_text    id  \
0                     <mismatch> a fictional superhero  4038   
1    <mismatch> the 39th President of the United St...  5468   
2                      <mismatch> Francisco Scaramanga  4973   
3           <mismatch> in the Battle of Bosworth Field  6578   
4    <mismatch> the 41st President of the United St...   439   
..                                                 ...   ...   
801                  <mismatch> on September 30 , 2016  3584   
802                        <mismatch> positive reviews   766   
803  <mismatch> under the care of his paternal uncl...  2660   
804  <mismatch> an Israeli-American media proprieto...  1770   
805                                     <mismatch> was  5413   

                                       predicted_label  \
0                                         mismatch> is   
1                       mismatch> of the United States   
2                       mismatch> Francisco Scaramanga   
3            mismatch> in the Battle of Bosworth Field   
4    mismatch> the 41st President of the United States   
..                                                 ...   
801                    mismatch> on September 30, 2016   
802                         mismatch> positive reviews   
803             mismatch> his paternal uncle Abu Talib   
804                      mismatch> an Israeli-American   
805                                      mismatch> was   

                                        original_label  
0                     <mismatch> a fictional superhero  
1    <mismatch> the 39th President of the United St...  
2                      <mismatch> Francisco Scaramanga  
3           <mismatch> in the Battle of Bosworth Field  
4    <mismatch> the 41st President of the United St...  
..                                                 ...  
801                  <mismatch> on September 30 , 2016  
802                        <mismatch> positive reviews  
803  <mismatch> under the care of his paternal uncl...  
804  <mismatch> an Israeli-American media proprieto...  
805                                     <mismatch> was  

[806 rows x 5 columns]

In [14]:
def process_output_r(x):
    x_split = x.split(' ')
    # ls = ['', '', '', '']
    ls = ['']
    itr = None
    for i in x_split:
        # if i == 'source>':
        #     itr = 0
        # elif i == 'relation>':
        #     itr = 1
        # elif i == 'target>':
        #     itr = 1
        if i == 'mismatch>':
            itr = 0
        elif itr is not None:
            ls[itr] += i + ' '
    return pd.Series([ls[0]])


def process_input_r(x):
    x_split = x.split(' ')
    # ls = ['', '', '', '']
    ls = ['']
    itr = None
    for i in x_split:
        # if i == 'source>':
        #     itr = 0
        # elif i == 'relation>':
        #     itr = 1
        # elif i == 'target>':
        #     itr = 1
        if i == '<mismatch>':
            itr = 0
        elif itr is not None:
            ls[itr] += i + ' '
    return pd.Series([ls[0]])

In [15]:
test_df[['pMismatch']] = test_df['predicted_label'].apply(process_output_r)
test_df[['Mismatch']] = test_df['original_label'].apply(process_input_r)

In [16]:
# df_test['predicted_label'] = predictions
# df_test['original_label'] = test_df['target_text']

In [17]:
test_df

source_text  \
0    <claim> John Stewart is a comic book villain.\...   
1    <claim> Jimmy Carter served as President of Co...   
2    <claim> Christopher Lee portrayed Hillary Clin...   
3    <claim> Richard III of England died in the Bat...   
4    <claim> George H. W. Bush served as the 39th p...   
..                                                 ...   
801  <claim> Denial (2016 film) was released in the...   
802  <claim> Star Wars: The Force Awakens received ...   
803  <claim> Muhammad was raised under the care of ...   
804  <claim> Haim Saban is exclusively an Italian-A...   
805  <claim> Nikola Tesla refused to be a physicist...   

                                           target_text    id  \
0                     <mismatch> a fictional superhero  4038   
1    <mismatch> the 39th President of the United St...  5468   
2                      <mismatch> Francisco Scaramanga  4973   
3           <mismatch> in the Battle of Bosworth Field  6578   
4    <mismatch> the 41st President of the United St...   439   
..                                                 ...   ...   
801                  <mismatch> on September 30 , 2016  3584   
802                        <mismatch> positive reviews   766   
803  <mismatch> under the care of his paternal uncl...  2660   
804  <mismatch> an Israeli-American media proprieto...  1770   
805                                     <mismatch> was  5413   

                                       predicted_label  \
0                                         mismatch> is   
1                       mismatch> of the United States   
2                       mismatch> Francisco Scaramanga   
3            mismatch> in the Battle of Bosworth Field   
4    mismatch> the 41st President of the United States   
..                                                 ...   
801                    mismatch> on September 30, 2016   
802                         mismatch> positive reviews   
803             mismatch> his paternal uncle Abu Talib   
804                      mismatch> an Israeli-American   
805                                      mismatch> was   

                                        original_label  \
0                     <mismatch> a fictional superhero   
1    <mismatch> the 39th President of the United St...   
2                      <mismatch> Francisco Scaramanga   
3           <mismatch> in the Battle of Bosworth Field   
4    <mismatch> the 41st President of the United St...   
..                                                 ...   
801                  <mismatch> on September 30 , 2016   
802                        <mismatch> positive reviews   
803  <mismatch> under the care of his paternal uncl...   
804  <mismatch> an Israeli-American media proprieto...   
805                                     <mismatch> was   

                                    pMismatch  \
0                                         is    
1                       of the United States    
2                       Francisco Scaramanga    
3            in the Battle of Bosworth Field    
4    the 41st President of the United States    
..                                        ...   
801                    on September 30, 2016    
802                         positive reviews    
803             his paternal uncle Abu Talib    
804                      an Israeli-American    
805                                      was    

                                              Mismatch  
0                               a fictional superhero   
1             the 39th President of the United States   
2                                Francisco Scaramanga   
3                     in the Battle of Bosworth Field   
4             the 41st President of the United States   
..                                                 ...  
801                            on September 30 , 2016   
802                                  positive reviews   
803    under the care of his paternal uncle Abu Talib   
804  an Israeli-American

In [18]:
def exact_match(orig, predicted):
    match = 0
    for i in range(len(orig)):
        if orig[i].strip() == predicted[i].strip():
            # print(orig[i], "&", predicted[i])
            match += 1
    return match/len(orig)

def partial_match(orig, predicted):
    match = 0
    for i in range(len(orig)):
        orig_split = orig[i].split()
        predicted_split = predicted[i].split()
        if len(orig_split) == 0:
            if len(predicted_split) == 0:
                match += 1
            else:
                match += 0
            continue
        ints = list(set(orig_split) & set(predicted_split))
        match += len(ints)/len(orig_split)
    return match/len(orig)

def IOU(orig, predicted):
    match = 0
    for i in range(len(orig)):
        orig_split = orig[i].split()
        predicted_split = predicted[i].split()
        if len(orig_split) == 0:
            if len(predicted_split) == 0:
                match += 1
            else:
                match += 0
            continue
        ints = list(set(orig_split) & set(predicted_split))
        match += len(ints)/(len(orig_split)+len(predicted_split)-len(ints))
    return match/len(orig)

In [19]:
# exact_match(list(test_df.pSource), list(test_df.Source)), partial_match(list(test_df.pSource), list(test_df.Source))

In [20]:
# exact_match(list(test_df.pRelation), list(test_df.Relation)), partial_match(list(test_df.pRelation), list(test_df.Relation))

In [21]:
# exact_match(list(test_df.pTarget), list(test_df.Target)), partial_match(list(test_df.pTarget), list(test_df.Target))

In [22]:
exact_match(list(test_df.Mismatch), list(test_df.pMismatch)), partial_match(list(test_df.Mismatch), list(test_df.pMismatch)), IOU(list(test_df.Mismatch), list(test_df.pMismatch))

(0.358560794044665, 0.5411275449448949, 0.5177683868643452)

In [23]:
test_df.to_csv(resultsfile)

In [24]:
join_df = test_df[["id","pMismatch"]]
res_df = df_test.join(join_df.set_index('id'), on='id')
res_df.to_csv(resultsfile)

# res_df